# Phân loại UCI News Aggregator dataset bằng Multinomial Naive Bayes
- Dataset: uci-news.csv (tab-separated)
- Chỉ sử dụng TITLE để phân loại CATEGORY

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Đọc dữ liệu

In [2]:
columns = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", 
           "STORY", "HOSTNAME", "TIMESTAMP"]

df = pd.read_csv("uci-news.csv", sep="\t", header=None, names=columns)
print("Kích thước dữ liệu:", df.shape)
df.head()

Kích thước dữ liệu: (422419, 8)


,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


## EDA cơ bản

In [3]:
print("Số lượng bản tin theo category:")
print(df["CATEGORY"].value_counts())

print("\nTỉ lệ phần trăm:")
print(df["CATEGORY"].value_counts(normalize=True))

Số lượng bản tin theo category:
CATEGORY
e    152469
b    115967
t    108344
m     45639
Name: count, dtype: int64

Tỉ lệ phần trăm:
CATEGORY
e    0.360943
b    0.274531
t    0.256485
m    0.108042
Name: proportion, dtype: float64


## Chuẩn bị dữ liệu train/test

In [4]:
X = df["TITLE"]
y = df["CATEGORY"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Xây dựng pipeline với CountVectorizer + Tfidf + MultinomialNB

In [5]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words="english")),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

## Đánh giá mô hình

In [6]:
y_pred = text_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9273116803181668

Classification Report:
               precision    recall  f1-score   support

           b       0.90      0.92      0.91     23193
           e       0.95      0.97      0.96     30494
           m       0.97      0.86      0.91      9128
           t       0.91      0.90      0.91     21669

    accuracy                           0.93     84484
   macro avg       0.93      0.91      0.92     84484
weighted avg       0.93      0.93      0.93     84484


Confusion Matrix:
 [[21230   497   124  1342]
 [  353 29675    52   414]
 [  533   493  7884   218]
 [ 1493   545    77 19554]]


## Nhận xét



* Mô hình **Multinomial Naive Bayes** đạt **độ chính xác tổng thể \~92.7%**, đây là mức khá cao cho một phương pháp baseline đơn giản.
* **Category `e` (entertainment)** được dự đoán rất tốt với **precision 0.95, recall 0.97**, gần như mô hình nhận diện chính xác hầu hết các mẫu.
* **Category `m` (health/medicine)** tuy precision cao (0.97) nhưng recall thấp hơn (0.86) → mô hình bỏ sót nhiều tin thuộc nhóm này.
* **Category `b` (business)** và **`t` (science/technology)** có kết quả cân bằng, nhưng vẫn còn một số nhầm lẫn qua lại (thể hiện rõ trong confusion matrix: nhiều mẫu `b` bị đoán thành `t` và ngược lại).
* Nhìn chung:

  * `e` là nhóm dễ phân loại nhất.
  * `m` là nhóm khó phân loại nhất (cần thêm đặc trưng hoặc xử lý nâng cao).
  * `b` và `t` có sự chồng lấn về ngữ nghĩa nên bị nhầm lẫn một phần.

🔎 Kết luận:
MultinomialNB là baseline nhanh, rẻ, cho kết quả **>90% accuracy**. Tuy nhiên, để cải thiện thêm, có thể:

* Dùng **n-grams** thay vì chỉ unigram trong `CountVectorizer`.
* Dùng mô hình mạnh hơn (Logistic Regression, Linear SVM).
* Áp dụng mô hình ngôn ngữ hiện đại (BERT/transformer).
